Control Flow Prediction
===

Process Prediction that focuses only on the control flow perspective of processes

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.eventlog import *
from exp.dl_utils import *
import editdistance as ed

# Import Data

In [3]:
ta=import_xes(untar_data(URLs.BPIC_2012)).traceAttributes

In [4]:
ta

,REG_DATE,AMOUNT_REQ
concept:name,,
173688,2011-10-01 00:38:44.546000+02:00,20000
173691,2011-10-01 08:08:58.256000+02:00,5000
173694,2011-10-01 08:10:30.287000+02:00,7000
173697,2011-10-01 08:11:08.865000+02:00,15000
173700,2011-10-01 08:15:39.894000+02:00,5000
...,...,...
214364,2012-02-29 23:22:24.570000+01:00,5000
214367,2012-02-29 23:28:41.098000+01:00,500
214370,2012-02-29 23:28:55.349000+01:00,20000


In [5]:
log=import_xes(untar_data(URLs.BPIC_2012)).events

In [6]:
log

,trace_id,event_id,org:resource,lifecycle:transition,concept:name,time:timestamp
0,173688,0,112,COMPLETE,A_SUBMITTED,2011-09-30 22:38:44.546000+00:00
1,173688,1,112,COMPLETE,A_PARTLYSUBMITTED,2011-09-30 22:38:44.880000+00:00
2,173688,2,112,COMPLETE,A_PREACCEPTED,2011-09-30 22:39:37.906000+00:00
3,173688,3,112,SCHEDULE,W_Completeren aanvraag,2011-09-30 22:39:38.875000+00:00
4,173688,4,None,START,W_Completeren aanvraag,2011-10-01 09:36:46.437000+00:00
...,...,...,...,...,...,...
262195,214376,1,112,COMPLETE,A_PARTLYSUBMITTED,2012-02-29 22:51:17.423000+00:00
262196,214376,2,112,SCHEDULE,W_Afhandelen leads,2012-02-29 22:52:01.287000+00:00
262197,214376,3,11169,START,W_Afhandelen leads,2012-03-01 08:26:46.736000+00:00
262198,214376,4,11169,COMPLETE,A_DECLINED,2012-03-01 08:27:37.118000+00:00


# Control Flow List

In [7]:
#export
class ListContainer():
    def __init__(self, items): self.items = listify(items)
    def __getitem__(self, idx):
        if isinstance(idx, (int,slice)): return self.items[idx]
        if isinstance(idx[0],bool):
            assert len(idx)==len(self) # bool mask
            return [o for m,o in zip(idx,self.items) if m]
        return [self.items[i] for i in idx]
    def __len__(self): return len(self.items)
    def __iter__(self): return iter(self.items)
    def __setitem__(self, i, o): self.items[i] = o
    def __delitem__(self, i): del(self.items[i])
    def __repr__(self):
        res = f'{self.__class__.__name__} ({len(self)} items)\n{self.items[:10]}'
        if len(self)>10: res = res[:-1]+ '...]'
        return res

In [8]:
#export
class ControlFlowList(ListContainer):

    def __init__(self,items,path='.'): 
        super().__init__(items)
        self.path=Path(path)
        
    @classmethod
    def from_df(cls,df,col='concept:name',trace_id='trace_id'):
        items=[]
        for n,g in df.groupby('trace_id'):
            items.append(list(g[col]))
        
        return cls(items)
    def new(self, items, cls=None):
        if cls is None: cls=self.__class__
        return cls(items, self.path)
    
    def get(self,o): return "->".join([str(x) for x in listify(self.items[o])])

    def __repr__(self):
        res = f'{self.__class__.__name__} ({len(self)} items)\n{[self.get(i) for i in range(2)]}'
        if len(self)>2: res = res[:-1]+ '...]'
        return res

In [9]:
cfl=ControlFlowList.from_df(log)
cfl

ControlFlowList (13087 items)
['A_SUBMITTED->A_PARTLYSUBMITTED->A_PREACCEPTED->W_Completeren aanvraag->W_Completeren aanvraag->A_ACCEPTED->O_SELECTED->A_FINALIZED->O_CREATED->O_SENT->W_Nabellen offertes->W_Completeren aanvraag->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->O_SENT_BACK->W_Valideren aanvraag->W_Nabellen offertes->W_Valideren aanvraag->A_REGISTERED->A_APPROVED->O_ACCEPTED->A_ACTIVATED->W_Valideren aanvraag', 'A_SUBMITTED->A_PARTLYSUBMITTED->A_PREACCEPTED->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->A_ACCEPTED->A_FINALIZED->O_SELECTED->O_CREATED->O_SENT->W_Nabellen offertes->W_Completeren aanvraag->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->O_SELECTED->O_CANCELLED->O_CREATED->O_SENT->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->O_SENT_BACK->W_Valideren aanvraag->W_Nabellen offertes->W_

In [10]:
cfl[0]

['A_SUBMITTED',
 'A_PARTLYSUBMITTED',
 'A_PREACCEPTED',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'A_ACCEPTED',
 'O_SELECTED',
 'A_FINALIZED',
 'O_CREATED',
 'O_SENT',
 'W_Nabellen offertes',
 'W_Completeren aanvraag',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'O_SENT_BACK',
 'W_Valideren aanvraag',
 'W_Nabellen offertes',
 'W_Valideren aanvraag',
 'A_REGISTERED',
 'A_APPROVED',
 'O_ACCEPTED',
 'A_ACTIVATED',
 'W_Valideren aanvraag']

# Splitting Data in Train, Test and Valid

In [11]:
#export
import random

In [12]:
#export

def random_splitter(fn, p_valid): return random.random() < p_valid
def split_by_func(items, f):
    mask = [f(o) for o in items]
    # `None` values will be filtered out
    f = [o for o,m in zip(items,mask) if m==False]
    t = [o for o,m in zip(items,mask) if m==True ]
    return f,t

class SplitData():
    def __init__(self, train, valid, test): self.train,self.valid,self.test = train,valid,test
    
    @classmethod
    def split_by_func(cls, il, f, Test=True):
        train, valid = map(il.new, split_by_func(il.items, f))
        test=None
        if Test:
            train, test = map(train.new, split_by_func(train.items, f))

        return cls(train,valid,test)

    def __repr__(self): return f'{self.__class__.__name__}\nTrain: {self.train}\nValid: {self.valid}\nTest: {self.test}\n'

In [13]:
sd=SplitData.split_by_func(cfl, partial(random_splitter, p_valid=0.1));sd

SplitData
Train: ControlFlowList (10540 items)
['A_SUBMITTED->A_PARTLYSUBMITTED->A_PREACCEPTED->W_Completeren aanvraag->W_Completeren aanvraag->A_ACCEPTED->O_SELECTED->A_FINALIZED->O_CREATED->O_SENT->W_Nabellen offertes->W_Completeren aanvraag->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->O_SENT_BACK->W_Valideren aanvraag->W_Nabellen offertes->W_Valideren aanvraag->A_REGISTERED->A_APPROVED->O_ACCEPTED->A_ACTIVATED->W_Valideren aanvraag', 'A_SUBMITTED->A_PARTLYSUBMITTED->A_DECLINED'...]
Valid: ControlFlowList (1312 items)
['A_SUBMITTED->A_PARTLYSUBMITTED->A_PREACCEPTED->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->W_Completeren aanvraag->A_ACCEPTED->A_FINALIZED->O_SELECTED->O_CREATED->O_SENT->W_Nabellen offertes->W_Completeren aanvraag->W_Nabellen offertes->O_SELECTED->O_CANCELLED->O_CREATED->O_SENT->W_Nabe

In [14]:
sd.train

ControlFlowList (10540 items)
['A_SUBMITTED->A_PARTLYSUBMITTED->A_PREACCEPTED->W_Completeren aanvraag->W_Completeren aanvraag->A_ACCEPTED->O_SELECTED->A_FINALIZED->O_CREATED->O_SENT->W_Nabellen offertes->W_Completeren aanvraag->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->O_SENT_BACK->W_Valideren aanvraag->W_Nabellen offertes->W_Valideren aanvraag->A_REGISTERED->A_APPROVED->O_ACCEPTED->A_ACTIVATED->W_Valideren aanvraag', 'A_SUBMITTED->A_PARTLYSUBMITTED->A_DECLINED'...]

# Encoding

## Tokenization

In [15]:
#export
UNK, PAD, BOT, EOT = "xxunk xxpad xxbot xxeot ".split()

def add_eot_bot(x): return  x + [EOT]
default_spec_tok = [UNK, PAD, BOT, EOT]
default_pre_rules = []
default_post_rules = [add_eot_bot]

In [16]:
#export
from concurrent.futures import ProcessPoolExecutor

def parallel(func, arr, max_workers=4):
    if max_workers<2: results = list(progress_bar(map(func, enumerate(arr)), total=len(arr)))
    else:
        with ProcessPoolExecutor(max_workers=max_workers) as ex:
            return list(progress_bar(ex.map(func, enumerate(arr)), total=len(arr)))
    if any([o is not None for o in results]): return results
    

In [17]:
#export
class TokenizeProcessor():
    def __init__(self, lang="en", chunksize=2000, pre_rules=None, post_rules=None, max_workers=4): 
        self.chunksize,self.max_workers = chunksize,max_workers        
        self.pre_rules  = default_pre_rules  if pre_rules  is None else pre_rules
        self.post_rules = default_post_rules if post_rules is None else post_rules

    def proc_chunk(self, args):
        i,chunk = args
        docs = [compose(t, self.pre_rules) for t in chunk]
        docs = [compose(t, self.post_rules) for t in docs]
        return docs

    def __call__(self, items): 
        toks = []
        if isinstance(items[0], Path): items = [read_file(i) for i in items]
        chunks = [items[i: i+self.chunksize] for i in (range(0, len(items), self.chunksize))]
        toks = parallel(self.proc_chunk, chunks, max_workers=self.max_workers)
        return sum(toks, [])
    
    def proc1(self, item): return self.proc_chunk([item])[0]
    

In [18]:
tp = TokenizeProcessor()

In [19]:
cfl.get(0)

'A_SUBMITTED->A_PARTLYSUBMITTED->A_PREACCEPTED->W_Completeren aanvraag->W_Completeren aanvraag->A_ACCEPTED->O_SELECTED->A_FINALIZED->O_CREATED->O_SENT->W_Nabellen offertes->W_Completeren aanvraag->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->W_Nabellen offertes->O_SENT_BACK->W_Valideren aanvraag->W_Nabellen offertes->W_Valideren aanvraag->A_REGISTERED->A_APPROVED->O_ACCEPTED->A_ACTIVATED->W_Valideren aanvraag'

In [20]:
' -> '.join(tp(cfl[:2])[0])

'A_SUBMITTED -> A_PARTLYSUBMITTED -> A_PREACCEPTED -> W_Completeren aanvraag -> W_Completeren aanvraag -> A_ACCEPTED -> O_SELECTED -> A_FINALIZED -> O_CREATED -> O_SENT -> W_Nabellen offertes -> W_Completeren aanvraag -> W_Nabellen offertes -> W_Nabellen offertes -> W_Nabellen offertes -> W_Nabellen offertes -> W_Nabellen offertes -> O_SENT_BACK -> W_Valideren aanvraag -> W_Nabellen offertes -> W_Valideren aanvraag -> A_REGISTERED -> A_APPROVED -> O_ACCEPTED -> A_ACTIVATED -> W_Valideren aanvraag -> xxeot'

## Numericalization

In [21]:
#export
import collections

class NumericalizeProcessor():
    def __init__(self, vocab=None, max_vocab=60000, min_freq=0,spec_token=None,unk_token=None): 
        self.vocab,self.max_vocab,self.min_freq = vocab,max_vocab,min_freq
        self.spec_token = default_spec_tok if spec_token is None else spec_token
        self.unk_token = UNK if unk_token is None else unk_token

    
    def __call__(self, items):
        #The vocab is defined on the first use.
        if self.vocab is None:
            freq = Counter(p for o in items for p in o)
            self.vocab = [o for o,c in freq.most_common(self.max_vocab) if c >= self.min_freq]
            for o in reversed(default_spec_tok):
                if o in self.vocab: self.vocab.remove(o)
                self.vocab.insert(0, o)
        if getattr(self, 'otoi', None) is None:
            self.otoi = collections.defaultdict(int,{v:k for k,v in enumerate(self.vocab)}) 
        return [self.proc1(o) for o in items]
    def proc1(self, item):  return [self.otoi.get(o, self.unk_token) for o in item]
    
    def deprocess(self, idxs):
        assert self.vocab is not None
        return [self.deproc1(idx) for idx in idxs]
    def deproc1(self, idx): return [self.vocab[i] for i in idx]

In [22]:
proc_tok,proc_num = TokenizeProcessor(),NumericalizeProcessor()

## Application of Processors and Labeling

In [23]:
cfl.items[2]

['A_SUBMITTED',
 'A_PARTLYSUBMITTED',
 'A_PREACCEPTED',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'A_ACCEPTED',
 'A_FINALIZED',
 'O_SELECTED',
 'O_CREATED',
 'O_SENT',
 'W_Nabellen offertes',
 'W_Completeren aanvraag',
 'W_Nabellen offertes',
 'O_SELECTED',
 'O_CANCELLED',
 'O_CREATED',
 'O_SENT',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'O_SELECTED',
 'O_CANCELLED',
 'O_CREATED',
 'O_SENT',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen o

In [24]:
#export

def _label_by_func(ds, f, cls=ControlFlowList): return cls([f(o) for o in ds.items], path=ds.path)

#This is a slightly different from what was seen during the lesson,
#   we'll discuss the changes in lesson 11
class LabeledData():
    def process(self, il, proc): return il.new(compose(il.items, proc))

    def __init__(self, x, y, proc_x=None, proc_y=None):
        self.x,self.y = self.process(x, proc_x),self.process(y, proc_y)
        self.proc_x,self.proc_y = proc_x,proc_y
        
    def __repr__(self): return f'{self.__class__.__name__}\nx: {self.x}\ny: {self.y}\n'
    def __getitem__(self,idx): return self.x[idx],self.y[idx]
    def __len__(self): return len(self.x)
    
    def x_obj(self, idx): return self.obj(self.x, idx, self.proc_x)
    def y_obj(self, idx): return self.obj(self.y, idx, self.proc_y)
    
    def obj(self, items, idx, procs):
        isint = isinstance(idx, int) or (isinstance(idx,torch.LongTensor) and not idx.ndim)
        item = items[idx]
        for proc in reversed(listify(procs)):
            item = proc.deproc1(item) if isint else proc.deprocess(item)
        return item

    @classmethod
    def label_by_func(cls, il, f, proc_x=None, proc_y=None):
        return cls(il, _label_by_func(il, f), proc_x=proc_x, proc_y=proc_y)

def label_by_func(sd, f, proc_x=None, proc_y=None):
    train = LabeledData.label_by_func(sd.train, f, proc_x=proc_x, proc_y=proc_y)
    valid = LabeledData.label_by_func(sd.valid, f, proc_x=proc_x, proc_y=proc_y)
    test =None if sd.test is None else LabeledData.label_by_func(sd.test, f, proc_x=proc_x, proc_y=proc_y)

    return SplitData(train,valid,test)

In [25]:
%time ll = label_by_func(sd, lambda x: 0, proc_x = [proc_tok,proc_num])

CPU times: user 325 ms, sys: 333 ms, total: 657 ms
Wall time: 786 ms


In [26]:
ll

SplitData
Train: LabeledData
x: ControlFlowList (10540 items)
['9->10->12->4->4->16->13->17->14->15->5->4->5->5->5->5->5->19->7->5->7->21->22->24->23->7->3', '9->10->11->3'...]
y: ControlFlowList (10540 items)
['0', '0'...]

Valid: LabeledData
x: ControlFlowList (1312 items)
['9->10->12->4->4->4->4->4->4->4->4->16->17->13->14->15->5->4->5->13->18->14->15->5->5->5->5->5->5->5->5->5->13->18->14->15->5->5->5->5->5->5->5->5->5->5->5->5->5->19->7->5->7->24->22->21->23->7->27->3', '9->10->11->3'...]
y: ControlFlowList (1312 items)
['0', '0'...]

Test: LabeledData
x: ControlFlowList (1235 items)
['9->10->12->4->4->4->4->16->17->13->14->15->5->4->5->5->5->13->18->14->15->5->5->5->5->5->19->7->5->7->7->7->7->7->24->22->21->23->7->3', '9->10->8->8->12->4->8->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->4->11->4->3'...]
y: ControlFlowList (1235 items)
['0', '0'...]


# Suffix Model Training
- [Link](http://134.96.72.195:8029/notebooks/fastai-course-v3/nbs/dl2/translation.ipynb)

In [93]:
def prepare_suffix_prediction(ds, test=False):
    x,y=[],[]
    
    for o in ds.items:
        if len(o)<=1: continue
        for i in range(1,len(o)-1):
            x.append(tensor(o[:i]))
            y.append(tensor(o[i:]))
    return x,y


In [94]:
train_ds=Dataset(*prepare_suffix_prediction(ll.train.x))
valid_ds=Dataset(*prepare_suffix_prediction(ll.valid.x))

In [96]:
bs=64
train_sampler = SortishSampler(train_ds.x, key=lambda t: len(train_ds[t][0]), bs=bs//2)
valid_sampler = SortSampler(valid_ds.x, key=lambda t: len(valid_ds[t][0]))


In [97]:
train_dl = DataLoader(train_ds, batch_size=bs, sampler=train_sampler, drop_last=True)
valid_dl = DataLoader(valid_ds, batch_size=bs, sampler=valid_sampler, drop_last=True)


# Language Model Training
- [Preprocessing and DataLoader](http://134.96.72.195:8029/notebooks/fastai-course-v3/nbs/dl2/12_text.ipynb)
- [Model](http://134.96.72.195:8029/notebooks/fastai-course-v3/nbs/dl2/12a_awd_lstm.ipynb)

## Language Model DataLoader and DataBunch

In [27]:
#export
class CF_LM_Dataset():
    def __init__(self, data, bs=64, bptt=70, shuffle=False):
        self.data,self.bs,self.bptt,self.shuffle = data,bs,bptt,shuffle
        total_len = sum([len(t) for t in data.x])
        self.n_batch = total_len // bs
        self.batchify()
    
    def __len__(self): return ((self.n_batch-1) // self.bptt) * self.bs
    
    def __getitem__(self, idx):
        source = self.batched_data[idx % self.bs]
        seq_idx = (idx // self.bs) * self.bptt
        return source[seq_idx:seq_idx+self.bptt],source[seq_idx+1:seq_idx+self.bptt+1]
    
    def batchify(self):
        texts = self.data.x
        if self.shuffle: texts = texts[torch.randperm(len(texts))]
        stream = torch.cat([tensor(t) for t in texts])
        self.batched_data = stream[:self.n_batch * self.bs].view(self.bs, self.n_batch)

In [28]:
#export
def get_lm_dls(train_ds, valid_ds, bs, bptt, **kwargs):
    return (DataLoader(CF_LM_Dataset(train_ds, bs, bptt, shuffle=True), batch_size=bs, **kwargs),
            DataLoader(CF_LM_Dataset(valid_ds, bs, bptt, shuffle=False), batch_size=2*bs, **kwargs))

def lm_databunchify(sd, bs, bptt, **kwargs):
    return DataBunch(*get_lm_dls(sd.train, sd.valid, bs, bptt, **kwargs))

In [29]:
dl = DataLoader(CF_LM_Dataset(ll.valid, shuffle=True), batch_size=64)

In [30]:
iter_dl = iter(dl)
xb,yb = next(iter_dl)


In [31]:
bs,bptt = 64,70
data = lm_databunchify(ll, bs, bptt)

In [32]:
# train_x,train_y=process_data_for_next_step_prediction(ll.train.x)
# valid_x,valid_y=process_data_for_next_step_prediction(ll.valid.x)
# 
# train_ds=Dataset(train_x,train_y)
# valid_ds=Dataset(valid_x,valid_y)
# 
# train_sampler = SortishSampler(train_ds.x, key=func,bs=bs)
# valid_sampler = SortSampler(valid_ds.x, key=func)
# 
# train_dl = DataLoader(train_ds, batch_size=bs, sampler=train_sampler, collate_fn=partial(pad_collate,pad_first=True))
# valid_dl = DataLoader(valid_ds, batch_size=bs*2, sampler=valid_sampler, collate_fn=partial(pad_collate,pad_first=True))

In [33]:
# data=DataBunch(train_dl,valid_dl)

## AWD_LSTM

In [34]:
#export
def dropout_mask(x, sz, p):
    return x.new(*sz).bernoulli_(1-p).div_(1-p)

In [35]:
x = torch.randn(10,10)
mask = dropout_mask(x, (10,10), 0.5); mask

tensor([[0., 2., 2., 2., 0., 2., 2., 0., 2., 2.],
        [2., 2., 2., 2., 2., 2., 2., 2., 0., 0.],
        [0., 2., 0., 2., 0., 2., 0., 2., 0., 0.],
        [2., 2., 0., 0., 2., 0., 2., 2., 0., 2.],
        [0., 0., 2., 2., 2., 2., 2., 2., 2., 2.],
        [2., 0., 2., 2., 0., 2., 0., 2., 2., 0.],
        [0., 0., 2., 2., 2., 2., 2., 0., 2., 2.],
        [0., 0., 2., 0., 0., 0., 0., 2., 0., 0.],
        [0., 0., 2., 2., 2., 2., 2., 0., 2., 0.],
        [2., 2., 2., 0., 2., 2., 0., 0., 2., 2.]])

In [36]:
(x*mask).std(),x.std()

(tensor(1.5723), tensor(1.0193))

In [37]:
#export
class RNNDropout(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p=p

    def forward(self, x):
        if not self.training or self.p == 0.: return x
        m = dropout_mask(x.data, (x.size(0), 1, x.size(2)), self.p)
        return x * m

In [38]:
dp = RNNDropout(0.3)
tst_input = torch.randn(3,3,7)
tst_input.shape, dp(tst_input).shape

(torch.Size([3, 3, 7]), torch.Size([3, 3, 7]))

In [39]:
#export
import warnings

WEIGHT_HH = 'weight_hh_l0'

class WeightDropout(nn.Module):
    def __init__(self, module, weight_p=[0.], layer_names=[WEIGHT_HH]):
        super().__init__()
        self.module,self.weight_p,self.layer_names = module,weight_p,layer_names
        for layer in self.layer_names:
            #Makes a copy of the weights of the selected layers.
            w = getattr(self.module, layer)
            self.register_parameter(f'{layer}_raw', nn.Parameter(w.data))
            self.module._parameters[layer] = F.dropout(w, p=self.weight_p, training=False)

    def _setweights(self):
        for layer in self.layer_names:
            raw_w = getattr(self, f'{layer}_raw')
            self.module._parameters[layer] = F.dropout(raw_w, p=self.weight_p, training=self.training)

    def forward(self, *args):
        self._setweights()
        with warnings.catch_warnings():
            #To avoid the warning that comes because the weights aren't flattened.
            warnings.simplefilter("ignore")
            return self.module.forward(*args)

In [40]:
module = nn.LSTM(5, 2)
dp_module = WeightDropout(module, 0.4)
getattr(dp_module.module, WEIGHT_HH)

Parameter containing:
tensor([[-0.1550, -0.1514],
        [ 0.6160,  0.3657],
        [ 0.6583, -0.6230],
        [-0.5370,  0.5531],
        [ 0.3305,  0.0401],
        [ 0.5291,  0.5339],
        [ 0.0644, -0.5049],
        [-0.1983,  0.1457]], requires_grad=True)

In [41]:
tst_input = torch.randn(4,20,5)
h = (torch.zeros(1,20,2), torch.zeros(1,20,2))
x,h = dp_module(tst_input,h)
getattr(dp_module.module, WEIGHT_HH)

tensor([[-0.0000, -0.0000],
        [ 0.0000,  0.0000],
        [ 1.0972, -1.0384],
        [-0.8950,  0.9218],
        [ 0.0000,  0.0669],
        [ 0.8819,  0.8899],
        [ 0.0000, -0.0000],
        [-0.3306,  0.0000]], grad_fn=<MulBackward0>)

In [42]:
#export
class EmbeddingDropout(nn.Module):
    "Applies dropout in the embedding layer by zeroing out some elements of the embedding vector."
    def __init__(self, emb, embed_p):
        super().__init__()
        self.emb,self.embed_p = emb,embed_p
        self.pad_idx = self.emb.padding_idx
        if self.pad_idx is None: self.pad_idx = -1

    def forward(self, words, scale=None):
        if self.training and self.embed_p != 0:
            size = (self.emb.weight.size(0),1)
            mask = dropout_mask(self.emb.weight.data, size, self.embed_p)
            masked_embed = self.emb.weight * mask
        else: masked_embed = self.emb.weight
        if scale: masked_embed.mul_(scale)
        return F.embedding(words, masked_embed, self.pad_idx, self.emb.max_norm,
                           self.emb.norm_type, self.emb.scale_grad_by_freq, self.emb.sparse)

In [43]:
enc = nn.Embedding(100, 7, padding_idx=1)
enc_dp = EmbeddingDropout(enc, 0.5)
tst_input = torch.randint(0,100,(8,))
enc_dp(tst_input)

tensor([[-2.8629, -1.4265, -1.6424, -0.5082, -1.2095,  1.6933, -2.0471],
        [-1.2942, -1.6450, -2.0416,  0.3578, -3.3985,  2.6225, -3.0477],
        [ 0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
        [ 0.6545,  1.4835,  0.9930, -1.9619, -1.7513, -1.4126, -0.8534],
        [-1.8351, -1.1467,  1.2831,  1.8301,  2.2569,  1.0970,  0.6257],
        [ 0.0000,  0.0000,  0.0000,  0.0000, -0.0000, -0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.0000, -0.0000,  0.0000, -0.0000,  0.0000],
        [-0.4324, -2.6123,  5.1308, -0.7716,  1.9230, -0.9969,  0.0887]],
       grad_fn=<EmbeddingBackward>)

In [44]:
#export
def to_detach(h):
    "Detaches `h` from its history."
    return h.detach() if type(h) == torch.Tensor else tuple(to_detach(v) for v in h)

In [45]:
#export
class AWD_LSTM(nn.Module):
    "AWD-LSTM inspired by https://arxiv.org/abs/1708.02182."
    initrange=0.1

    def __init__(self, vocab_sz, emb_sz, n_hid, n_layers, pad_token,
                 hidden_p=0.2, input_p=0.6, embed_p=0.1, weight_p=0.5):
        super().__init__()
        self.bs,self.emb_sz,self.n_hid,self.n_layers = 1,emb_sz,n_hid,n_layers
        self.emb = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_token)
        self.emb_dp = EmbeddingDropout(self.emb, embed_p)
        self.rnns = [nn.LSTM(emb_sz if l == 0 else n_hid, (n_hid if l != n_layers - 1 else emb_sz), 1,
                             batch_first=True) for l in range(n_layers)]
        self.rnns = nn.ModuleList([WeightDropout(rnn, weight_p) for rnn in self.rnns])
        self.emb.weight.data.uniform_(-self.initrange, self.initrange)
        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(n_layers)])

    def forward(self, input):
        bs,sl = input.size()
        if bs!=self.bs:
            self.bs=bs
            self.reset()
        raw_output = self.input_dp(self.emb_dp(input))
        new_hidden,raw_outputs,outputs = [],[],[]
        for l, (rnn,hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            raw_output, new_h = rnn(raw_output, self.hidden[l])
            new_hidden.append(new_h)
            raw_outputs.append(raw_output)
            if l != self.n_layers - 1: raw_output = hid_dp(raw_output)
            outputs.append(raw_output) 
        self.hidden = to_detach(new_hidden)
        return raw_outputs, outputs

    def _one_hidden(self, l):
        "Return one hidden state."
        nh = self.n_hid if l != self.n_layers - 1 else self.emb_sz
        return next(self.parameters()).new(1, self.bs, nh).zero_()

    def reset(self):
        "Reset the hidden states."
        self.hidden = [(self._one_hidden(l), self._one_hidden(l)) for l in range(self.n_layers)]

In [46]:
#export
class LinearDecoder(nn.Module):
    def __init__(self, n_out, n_hid, output_p, tie_encoder=None, bias=True):
        super().__init__()
        self.output_dp = RNNDropout(output_p)
        self.decoder = nn.Linear(n_hid, n_out, bias=bias)
        if bias: self.decoder.bias.data.zero_()
        if tie_encoder: self.decoder.weight = tie_encoder.weight
        else: init.kaiming_uniform_(self.decoder.weight)

    def forward(self, input):
        raw_outputs, outputs = input
        output = self.output_dp(outputs[-1]).contiguous()
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded, raw_outputs, outputs

In [47]:
#export
class SequentialRNN(nn.Sequential):
    "A sequential module that passes the reset call to its children."
    def reset(self):
        for c in self.children():
            if hasattr(c, 'reset'): c.reset()

In [48]:
#export
def get_language_model(vocab_sz, emb_sz, n_hid, n_layers, pad_token, output_p=0.4, hidden_p=0.2, input_p=0.6, 
                       embed_p=0.1, weight_p=0.5, tie_weights=True, bias=True):
    rnn_enc = AWD_LSTM(vocab_sz, emb_sz, n_hid=n_hid, n_layers=n_layers, pad_token=pad_token,
                       hidden_p=hidden_p, input_p=input_p, embed_p=embed_p, weight_p=weight_p)
    enc = rnn_enc.emb if tie_weights else None
    return SequentialRNN(rnn_enc, LinearDecoder(vocab_sz, emb_sz, output_p, tie_encoder=enc, bias=bias))

In [49]:
tok_pad = proc_num.vocab.index(PAD)

In [50]:
tst_model = get_language_model(len(proc_num.vocab), 300, 300, 2, tok_pad)


In [51]:
z=tst_model(xb)
z

(tensor([[ 0.0005, -0.0068,  0.0202,  ...,  0.0261,  0.0133,  0.0154],
         [-0.0010, -0.0162,  0.0332,  ...,  0.0396,  0.0166,  0.0187],
         [-0.0019, -0.0252,  0.0400,  ...,  0.0464,  0.0155,  0.0189],
         ...,
         [ 0.0126, -0.0105,  0.0271,  ...,  0.0282,  0.0166, -0.0482],
         [ 0.0096, -0.0148,  0.0320,  ...,  0.0371,  0.0121, -0.0475],
         [ 0.0089, -0.0183,  0.0362,  ...,  0.0447,  0.0083, -0.0483]],
        grad_fn=<AddmmBackward>),
 [tensor([[[ 0.0012,  0.0156, -0.0359,  ...,  0.0269,  0.0184, -0.0230],
           [-0.0100,  0.0013, -0.0131,  ...,  0.0267,  0.0209, -0.0349],
           [-0.0137, -0.0033, -0.0040,  ...,  0.0243,  0.0198, -0.0398],
           ...,
           [ 0.0126,  0.0072,  0.0232,  ...,  0.0158,  0.0098, -0.0685],
           [ 0.0015,  0.0026,  0.0080,  ...,  0.0145,  0.0091, -0.0512],
           [ 0.0216,  0.0082,  0.0119,  ...,  0.0173,  0.0179, -0.0763]],
  
          [[ 0.0029, -0.0122,  0.0124,  ...,  0.0410,  0.0002, -0.0

In [52]:
# export
def cross_entropy_flat(input, target):
    bs,sl = target.size()
    return F.cross_entropy(input.view(bs * sl, -1), target.view(bs * sl))

def accuracy_flat(input, target):
    bs,sl = target.size()
    return accuracy(input.view(bs * sl, -1), target.view(bs * sl))

In [53]:
decoded, raw_outputs, outputs = z
accuracy_flat(decoded,yb)


tensor(0.0154)

In [54]:
cross_entropy_flat(decoded,yb)

tensor(3.3371, grad_fn=<NllLossBackward>)

## Training Loop with AWD-LSTM

In [55]:
#export
class GradientClipping(Callback):
    def __init__(self, clip=None): self.clip = clip
    def after_backward(self):
        if self.clip:  nn.utils.clip_grad_norm_(self.run.model.parameters(), self.clip)

In [56]:
#export
class RNNTrainer(Callback):
    def __init__(self, α, β): self.α,self.β = α,β
    
    def after_pred(self):
        #Save the extra outputs for later and only returns the true output.
        self.raw_out,self.out = self.pred[1],self.pred[2]
        self.run.pred = self.pred[0]
    
    def after_loss(self):
        #AR and TAR
        if self.α != 0.:  self.run.loss += self.α * self.out[-1].float().pow(2).mean()
        if self.β != 0.:
            h = self.raw_out[-1]
            if len(h)>1: self.run.loss += self.β * (h[:,1:] - h[:,:-1]).float().pow(2).mean()
                
    def begin_epoch(self):
        pass
        #Shuffle the texts at the beginning of the epoch
        #if hasattr(self.dl.dataset, "batchify"): self.dl.dataset.batchify()

In [57]:
#export
class CudaCallback(Callback):
    def begin_fit(self): self.model.cuda()
    def begin_batch(self): self.run.xb,self.run.yb = self.xb.cuda(),self.yb.cuda()

In [58]:
emb_sz, nh, nl = 300, 300, 2
model = get_language_model(len(proc_num.vocab), emb_sz, nh, nl, tok_pad, input_p=0.6, output_p=0.4, weight_p=0.5, 
                           embed_p=0.1, hidden_p=0.2)

In [59]:
cbs = [partial(AvgStatsCallback,accuracy_flat),
       CudaCallback, Recorder,
       partial(GradientClipping, clip=0.1),
       partial(RNNTrainer, α=2., β=1.),
       ProgressBarCallback]

In [60]:
learn = Learner(model, data, cross_entropy_flat, lr=5e-3, cb_funcs=cbs, opt_func=adam_opt())

In [ ]:
learn.fit(50)

epoch,train_loss,train_accuracy_flat,valid_loss,valid_accuracy_flat,time
0,1.674799,0.519169,0.823884,0.751711,00:01
1,0.723656,0.772659,0.543339,0.823772,00:01
2,0.565362,0.815520,0.506282,0.829502,00:01
3,0.519134,0.824034,0.471342,0.835007,00:01
4,0.474374,0.837031,0.461097,0.841295,00:01
5,0.450524,0.838402,0.450893,0.839211,00:01
6,0.436042,0.842543,0.451228,0.839955,00:01
7,0.425455,0.843112,0.447866,0.841257,00:01
8,0.436071,0.839636,0.441456,0.840997,00:01
9,0.418633,0.843212,0.435832,0.842448,00:01


# Testing

In [ ]:
sd.test

## Next Step Prediction

In [63]:
ll.test.x

ControlFlowList (1169 items)
['9->10->8->8->11->8->3', '9->10->8->8->11->8->3'...]

In [64]:
#export
from torch import LongTensor,FloatTensor

In [65]:
#export
def process_data_for_next_step_prediction(test,startIndex=1):
    xs,ys=[],[]
    for trace in progress_bar(test.items):
        for i in range(startIndex,len(listify(trace))):
            x,y=[],[]
            xs.append(tensor(trace[:i]))
            ys.append(tensor(trace[i]))
    return xs,tensor(ys)

In [66]:
x,y=process_data_for_next_step_prediction(ll.test.x)

In [ ]:
#export
def func(t):
    return len(pd_data[int(t)][0])

In [67]:
pd_data=Dataset(x,y)

In [68]:
pd_data.x

[tensor([9]),
 tensor([ 9, 10]),
 tensor([ 9, 10,  8]),
 tensor([ 9, 10,  8,  8]),
 tensor([ 9, 10,  8,  8, 11]),
 tensor([ 9, 10,  8,  8, 11,  8]),
 tensor([9]),
 tensor([ 9, 10]),
 tensor([ 9, 10,  8]),
 tensor([ 9, 10,  8,  8]),
 tensor([ 9, 10,  8,  8, 11]),
 tensor([ 9, 10,  8,  8, 11,  8]),
 tensor([9]),
 tensor([ 9, 10]),
 tensor([ 9, 10, 12]),
 tensor([ 9, 10, 12,  4]),
 tensor([ 9, 10, 12,  4,  4]),
 tensor([ 9, 10, 12,  4,  4,  4]),
 tensor([ 9, 10, 12,  4,  4,  4,  4]),
 tensor([ 9, 10, 12,  4,  4,  4,  4, 16]),
 tensor([ 9, 10, 12,  4,  4,  4,  4, 16, 13]),
 tensor([ 9, 10, 12,  4,  4,  4,  4, 16, 13, 17]),
 tensor([ 9, 10, 12,  4,  4,  4,  4, 16, 13, 17, 14]),
 tensor([ 9, 10, 12,  4,  4,  4,  4, 16, 13, 17, 14, 15]),
 tensor([ 9, 10, 12,  4,  4,  4,  4, 16, 13, 17, 14, 15,  5]),
 tensor([ 9, 10, 12,  4,  4,  4,  4, 16, 13, 17, 14, 15,  5,  4]),
 tensor([ 9, 10, 12,  4,  4,  4,  4, 16, 13, 17, 14, 15,  5,  4,  5]),
 tensor([ 9, 10, 12,  4,  4,  4,  4, 16, 13, 17, 14, 15,  

In [70]:
#export
from torch.utils.data import Sampler

class SortSampler(Sampler):
    def __init__(self, data_source, key): self.data_source,self.key = data_source,key
    def __len__(self): return len(self.data_source)
    def __iter__(self):
        return iter(sorted(list(range(len(self.data_source))), key=self.key, reverse=True))

In [71]:
#export
class SortishSampler(Sampler):
    def __init__(self, data_source, key, bs):
        self.data_source,self.key,self.bs = data_source,key,bs

    def __len__(self) -> int: return len(self.data_source)

    def __iter__(self):
        idxs = torch.randperm(len(self.data_source))
        megabatches = [idxs[i:i+self.bs*50] for i in range(0, len(idxs), self.bs*50)]
        sorted_idx = torch.cat([tensor(sorted(s, key=self.key, reverse=True)) for s in megabatches])
        batches = [sorted_idx[i:i+self.bs] for i in range(0, len(sorted_idx), self.bs)]
        max_idx = torch.argmax(tensor([self.key(ck[0]) for ck in batches]))  # find the chunk with the largest key,
        batches[0],batches[max_idx] = batches[max_idx],batches[0]            # then make sure it goes first.
        batch_idxs = torch.randperm(len(batches)-2)
        sorted_idx = torch.cat([batches[i+1] for i in batch_idxs]) if len(batches) > 1 else LongTensor([])
        sorted_idx = torch.cat([batches[0], sorted_idx, batches[-1]])
        return iter(sorted_idx)

In [71]:
#export
def pad_collate(samples, pad_idx=1, pad_first=False):
    max_len = max([len(s[0]) for s in samples])
    res = torch.zeros(len(samples), max_len).long() + pad_idx
    for i,s in enumerate(samples):
        if pad_first: res[i, -len(s[0]):] = LongTensor(s[0])
        else:         res[i, :len(s[0]) ] = LongTensor(s[0])
    return res, tensor([s[1] for s in samples])

In [72]:
#export
def func(t):
    return len(pd_data[int(t)][0])

In [73]:
train_sampler = SortishSampler(pd_data.x, key=func,bs=bs)
test_sampler = SortSampler(pd_data.x, key=func)


In [28]:
pad_collate??

Object `pad_collate` not found.


In [74]:
test_dl = DataLoader(pd_data, batch_size=bs*64, sampler=test_sampler, collate_fn=partial(pad_collate,pad_first=True))

In [75]:
#export
def predict_next_step(learner, test_dl):
    iter_dl = iter(test_dl)
    learner.model.cuda()
    learner.model.eval()
    acc4batch = []
    for x, y in progress_bar(iter_dl):
        x, y = x.cuda(), y.cuda()
        learner.model.reset()
        pred = learner.model(x)
        acc4batch.append((torch.argmax(learner.model(x)[0].view(*x.shape,-1)[:,-1,:], dim=1)==y).float().mean().cpu())
    return np.mean(acc4batch)

In [76]:
predict_next_step(learn, test_dl)

0.8548791

In [77]:
# %%time
# iter_dl = iter(test_dl)
# learn.model.cuda()
# learn.model.eval()
# acc4batch=[]
# for x,y in progress_bar(iter_dl):
#     x,y=x.cuda(),y.cuda()
#     learn.model.reset()
#     pred=learn.model(x)
#     acc4batch.append((torch.argmax(learn.model(x)[0].view(*x.shape,-1)[:,-1,:], dim=1)==y).float().mean().cpu())
# np.mean(acc4batch)

In [78]:
# np.mean(acc4batch)

## Suffix Prediction

Since the model is a LM we need to sample data to get the final suffix prediction

In [79]:
#export
def process_data_for_suffix_prediction(test,startIndex=1):
    xs,ys=[],[]
    for trace in progress_bar(test.items):
        for i in range(startIndex,len(listify(trace))):
            x,y=[],[]
            xs.append(tensor(trace[:i]))
            ys.append(tensor(trace[i:]))
    return xs,ys

In [80]:
xs,ys=process_data_for_suffix_prediction(ll.test.x)

In [81]:
pd_data=Dataset(xs,ys)

In [82]:
xs[0:10]

[tensor([9]),
 tensor([ 9, 10]),
 tensor([ 9, 10,  8]),
 tensor([ 9, 10,  8,  8]),
 tensor([ 9, 10,  8,  8, 11]),
 tensor([ 9, 10,  8,  8, 11,  8]),
 tensor([9]),
 tensor([ 9, 10]),
 tensor([ 9, 10,  8]),
 tensor([ 9, 10,  8,  8])]

In [83]:
ys[0:10]

[tensor([10,  8,  8, 11,  8,  3]),
 tensor([ 8,  8, 11,  8,  3]),
 tensor([ 8, 11,  8,  3]),
 tensor([11,  8,  3]),
 tensor([8, 3]),
 tensor([3]),
 tensor([10,  8,  8, 11,  8,  3]),
 tensor([ 8,  8, 11,  8,  3]),
 tensor([ 8, 11,  8,  3]),
 tensor([11,  8,  3])]

In [84]:
test_sampler = SortSampler(pd_data.x, key=func)


In [85]:
#export
def pad_collate_sp(samples, pad_idx=1):
    max_len = max([len(s[0]) for s in samples])
    res_x = torch.zeros(len(samples), max_len).long() + pad_idx
    for i,s in enumerate(samples): res_x[i, -len(s[0]):] = LongTensor(s[0])
    max_len = max([len(s[1]) for s in samples])
    res_y = torch.zeros(len(samples), max_len).long() + pad_idx
    for i,s in enumerate(samples): res_y[i, :len(s[1]) ] = LongTensor(s[1])
    return res_x,res_y

In [86]:
test_dl = DataLoader(pd_data, batch_size=bs*8, sampler=test_sampler, collate_fn=pad_collate_sp)

In [87]:
iter_dl=iter(test_dl)
a,b=next(iter_dl)


In [88]:
a

tensor([[ 9, 10,  8,  ..., 22, 24,  7],
        [ 1,  9, 10,  ..., 23, 22, 24],
        [ 1,  1,  9,  ..., 21, 23, 22],
        ...,
        [ 1,  1,  1,  ...,  6,  6,  6],
        [ 1,  1,  1,  ...,  7,  6,  6],
        [ 1,  1,  1,  ...,  6,  6,  6]])

In [89]:
b

tensor([[ 3,  1,  1,  ...,  1,  1,  1],
        [ 7,  3,  1,  ...,  1,  1,  1],
        [24,  7,  3,  ...,  1,  1,  1],
        ...,
        [ 6,  6,  6,  ..., 24,  7,  3],
        [ 6,  6,  6,  ...,  1,  1,  1],
        [ 6,  7,  6,  ...,  1,  1,  1]])

In [102]:
#export
from pyxdameraulevenshtein import damerau_levenshtein_distance, normalized_damerau_levenshtein_distance

In [103]:
#export
def suffix_measure(x, y, x2, res):
    for k in range(len(y)):
        b = y[k]
        h = b!=1
        b = b[h].clone()
        a = x2[:, x.shape[1]:][k]
        a = a[h].clone()
        a_res = a.cpu().numpy()
        b_res = b.cpu().numpy()
        
        res.append(1-normalized_damerau_levenshtein_distance(list(a_res), list(b_res)))

In [104]:
#export
def predict_suffix(learner, test_dl):
    iter_dl = iter(test_dl)
    res = []
    learner.model.cuda()
    learner.model.eval()
    # preds4batch = []

    for x, y in progress_bar(iter_dl):
        learner.model.reset()
        x2 = x.clone().cuda()
        y = y.cuda()
        for i in range(y.shape[1]):
            x.shape, y.shape
            pred = torch.argmax(learner.model(x2)[0].view(*x2.shape,-1)[:,-1,:], dim=1)
            x2 = torch.cat([x2, pred[:, None]], dim=1)
        #preds4batch.append(x2[:,x.shape[1]:])
        suffix_measure(x, y, x2, res)
    return np.mean(res)

In [100]:
predict_suffix(learn, test_dl)

0.48026946695909745

In [101]:
# res=[]
# suffix_measure(x,y,x2,res)
# np.mean(res)

In [94]:
len(pd_data.x)

24183

In [95]:
len(res)

119

In [105]:
! /home/lahann/anaconda3/envs/fastpm/bin/python notebook2script.py 12_fastpm_control_flow_prediction.ipynb

Converted 12_fastpm_control_flow_prediction.ipynb to exp/control_flow_prediction.py
